In [ ]:
import torch
import torchvision

In [ ]:
print("torch:", torch.__version__)

torch: 1.12.1+cu113


In [ ]:
# loading the datasets
from torchvision.datasets import MNIST

In [ ]:
data_train = MNIST(root="./", download=True, train=True)
data_test = MNIST(root="./", download=True, train=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
data_train[10][0]

In [ ]:
data_train[10][1]

3

In [ ]:
type(data_train[10][0])

PIL.Image.Image

In [ ]:
print(data_train[0][0].size)

(28, 28)


In [ ]:
from torchvision.transforms import Lambda, ToTensor, Compose
transform = Compose([
    ToTensor(),
    Lambda(lambda image: image/255),
    Lambda(lambda image: image.view(784))
])

data_train = MNIST(root="./", download=True, train=True, transform = transform)
data_test = MNIST(root="./", download=True, train=False, transform = transform)


In [ ]:
data_train[0][0].shape

torch.Size([784])

In [ ]:
# Build the model
from torch import nn, optim

class Model(nn.Module):
  def __init__(self, sizes):   # [128,64,10]
    super().__init__()
    self.hidden_layer_1 = nn.Linear(784, sizes[0])  # 784,128
    self.act1 = nn.Sigmoid()
    self.hidden_layer_2 = nn.Linear(sizes[0], sizes[1])   # 128,64
    self.act2 = nn.Sigmoid()
    self.output_layer = nn.Linear(sizes[1], sizes[2])   # 64, 10
    self.act3 = nn.Softmax(dim=1)

    self.loss = nn.CrossEntropyLoss()
    self.optimizer = optim.Adam(self.parameters())

  def forward(self, inputs):
    x = self.hidden_layer_1(inputs)
    x = self.act1(x)
    x = self.hidden_layer_2(x)
    x = self.act2(x)
    x = self.output_layer(x)
    x = self.act3(x)
    return x

  def fit(self, X,Y):
    self.optimizer.zero_grad()
    y_pred = self.forward(X)
    loss = self.loss(y_pred, Y)
    loss.backward()   # calculation of gradients - backpropagation
    self.optimizer.step()   # updating the weights
    return loss.item()

  def predict(self,X):
    with torch.no_grad():
      return torch.argmax(self.forward(X), axis=1)
  
  def evaluate(self,test_dataloader):
    correct = 0
    for X,Y in test_dataloader:
      y_pred = self.predict(X)
      correct += (Y == y_pred).sum()   # batch
    acc = correct / (len(test_dataloader)* 16)
    print(f"Accuracy: {acc}")






In [ ]:
model = Model([128,64,10])

In [ ]:
from torch.utils.data import DataLoader
BATCH = 16
train_dataloader = DataLoader(data_train, batch_size=BATCH, shuffle=True)
test_dataloader = DataLoader(data_test, batch_size=BATCH, shuffle=True)

In [ ]:
from tqdm import tqdm
epochs = 10
for i in range(epochs):
  total_loss = 0
  for X,Y in tqdm(train_dataloader, desc = f"Fitting epoch {i}"):
    loss = model.fit(X,Y)
    total_loss += loss
  avg_loss = total_loss/len(train_dataloader)
  print(f"Epoch {i}: {avg_loss:.4f}")

Fitting epoch 0: 100%|██████████| 3750/3750 [00:13<00:00, 273.97it/s]


Epoch 0: 2.2035


Fitting epoch 1: 100%|██████████| 3750/3750 [00:12<00:00, 311.83it/s]


Epoch 1: 2.0118


Fitting epoch 2: 100%|██████████| 3750/3750 [00:12<00:00, 302.60it/s]


...

Epoch 7: 1.7078


Fitting epoch 8: 100%|██████████| 3750/3750 [00:12<00:00, 303.62it/s]


Epoch 8: 1.6965


Fitting epoch 9: 100%|██████████| 3750/3750 [00:11<00:00, 314.98it/s]

Epoch 9: 1.6873


In [ ]:
model.evaluate(test_dataloader)

Accuracy: 0.791700005531311


In [ ]:
path = "./mnist_model.pth"
torch.save(model.state_dict(), path)  # only weights


In [ ]:
model_2 = Model([128,64,10])
model_2.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
model_2.evaluate(test_dataloader)

Accuracy: 0.791700005531311


In [ ]:
path = "./full_model.pth"
torch.save(model, path)  # full model with arch and weights


In [ ]:
model_3 = torch.load(path)

In [ ]:
model_3.evaluate(test_dataloader)

Accuracy: 0.791700005531311


In [ ]:
for X,Y in test_dataloader:
  y_pred = model.predict(X)
  print(Y, y_pred)
  print(Y==y_pred)
  break

tensor([5, 7, 0, 2, 7, 7, 7, 8, 6, 2, 6, 3, 2, 4, 4, 5]) tensor([3, 9, 4, 3, 7, 7, 7, 2, 6, 3, 6, 3, 2, 4, 9, 5])
tensor([False, False, False, False,  True,  True,  True, False,  True, False,
         True,  True,  True,  True, False,  True])
